### Requirements/Libraries


In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON

### Query local Playground dataset

In [41]:
g = Graph()
g.parse("playground.ttl", format="ttl")

print("Loaded '" + str(len(g)) + "' triples.")

print("Query results (Females):")

qres = g.query(
"""SELECT ?thing ?name where {
  ?thing tto:sex "female" .
  ?thing dbp:name ?name .
}""")

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is female with name '%s'" % (str(row.thing),str(row.name)))
    
    
qres1 = g.query(
"""SELECT ?grandparent WHERE {
    ttr:Eve dbo:parent/dbo:parent ?grandparent .
}
""")

for row in qres1:
    print("Eve's Grandfather is '%s'" % (str(row.grandparent)))
    
qres2 = g.query(
"""SELECT ?dogs ?color ?sex WHERE {
    ?dogs rdf:type tto:Dog ;
        tto:color ?color ;
        tto:sex ?sex .
}
""")

for row in qres2:
    print("%s is a dog that is '%s' in color and who's sex is '%s'" % 
          (str(row.dogs),str(row.color),str(row.sex)))
    
    
qres3 = g.query(
"""SELECT ?pet ?owner WHERE {
    ?pet a [rdfs:subClassOf tto:Animal] .
    OPTIONAL {?owner tto:pet ?pet}
}
""")

for row in qres3:
    print("%s has owner '%s'" % (str(row.pet),str(row.owner)))

    
qres4 = g.query(
    """SELECT ?peo ?gen ?bir_dat WHERE {
     ?peo rdf:type dbo:Person ;
        tto:sex ?gen ;
        dbp:birthDate ?bir_dat .
}
ORDER BY ASC(?bir_dat) ?gen 
"""
)

for row in qres4:
    print("%s was born '%s' and gender is '%s'" % 
          (str(row.peo), str(row.bir_dat), str(row.gen)))

Loaded '73' triples.
Query results (Females):
http://example.org/tuto/resource#Eve is female with name 'Eve'
http://example.org/tuto/resource#LunaCat is female with name 'Luna'
Eve's Grandfather is 'http://example.org/tuto/resource#John'
http://example.org/tuto/resource#RexDog is a dog that is 'brown' in color and who's sex is 'male'
http://example.org/tuto/resource#LunaCat has owner 'http://example.org/tuto/resource#John'
http://example.org/tuto/resource#TomCat has owner 'http://example.org/tuto/resource#John'
http://example.org/tuto/resource#RexDog has owner 'http://example.org/tuto/resource#William'
http://example.org/tuto/resource#SnuffMonkey has owner 'None'
http://example.org/tuto/resource#John was born '1942-02-02' and gender is 'male'
http://example.org/tuto/resource#William was born '1978-07-20' and gender is 'male'
http://example.org/tuto/resource#Eve was born '2006-11-03' and gender is 'female'


### Query local Nobel Prize dataset

In [46]:
g = Graph()
g.parse("../files/nobelprize_kg.nt", format="nt")
  
print("Loaded '" + str(len(g)) + "' triples.")

#Query with prefixes 
nobelprize_query = """
PREFIX nobel: <http://data.nobelprize.org/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?name_laur WHERE { 
?laur rdf:type nobel:Laureate . 
?laur rdfs:label ?name_laur . 
?laur foaf:gender "female" . }
"""

qres = g.query(nobelprize_query)

print("Female laureates (printing 10):")
i=0
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    i+=1
    print("'%s'" % (str(row.name_laur))) #Same name as in sparql query SELECT variable
    if i==10:
        break
        
        
nbq1 = """
PREFIX nobel: <http://data.nobelprize.org/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?laur ?ctry WHERE {
    ?laur rdf:type nobel:Laureate .
    ?laur rdfs:label ?label .
    ?laur dbpedia-owl:birthPlace nobel-country:United_Kingdom .
} 
"""

qres10 = g.query(nbq1)

print("laureates (printing 10):")
i=0
for row in qres10:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    i+=1
    print("'%s'" % (str(row.laur))) #Same name as in sparql query SELECT variable
    if i==10:
        break

Loaded '85797' triples.
Female laureates (printing 10):
'Marie Curie, née Sklodowska'
'Maria Goeppert Mayer'
'Irène Joliot-Curie'
'Dorothy Crowfoot Hodgkin'
'Gerty Theresa Cori, née Radnitz'
'Rosalyn Yalow'
'Barbara McClintock'
'Rita Levi-Montalcini'
'Gertrude B. Elion'
'Christiane Nüsslein-Volhard'


Exception: Unknown namespace prefix : dbpedia-owl

### Query remote dataset via SPARQL Enpoint

In [6]:
endpoint_url = "http://dbpedia.org/sparql"
sparqlw = SPARQLWrapper(endpoint_url)
sparqlw.setReturnFormat(JSON)


query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX dbo: <http://dbpedia.org/ontology/> 
SELECT DISTINCT ?x WHERE { 
?jd foaf:name \"Johnny Depp\"@en . 
?m dbo:starring ?jd .
?m dbo:starring ?other . 
?other foaf:name ?x . 
FILTER (STR(?x)!=\"Johnny Depp\")
} 
ORDER BY ?x
"""

sparqlw.setQuery(query)

results = sparqlw.query().convert()

#Prints JSON file
#print(results)   

print("Stars co-starring Johnny Depp (printing 10):")
i=0
for result in results["results"]["bindings"]:
    i+=1
    #Prints individual results 
    print(result["x"]["value"])
    if i==10:
        break

Stars co-starring Johnny Depp (printing 10):

Abigail Breslin
Aidan Quinn
Al Pacino
Alain Chabat
Alan Arkin
Albert Tsai
Alfred Molina
Alison Sudol
Amanda Wyss
